# 第4回講義 宿題

## 課題. MNISTデータセットを多層パーセプトロン(MLP)で学習せよ

### 注意
- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください

- MLPの実装にTheanoなどのライブラリを使わないでください

### ヒント
- 出力yはone-of-k表現
- 最終層の活性化関数はソフトマックス関数, 誤差関数は多クラス交差エントロピー
- 最終層のデルタは教科書参照

次のコードが**事前**に実行されます


```python
from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=??) # random_stateはこちらで与えます
```

次のセルのhomework関数を完成させて提出してください
- パッケージのインポートなど、必要な物はすべて書いてください

In [18]:
def homework(train_X, test_X, train_y):
    # WRITE ME
    
    one_hot_y = np.zeros((train_y.size, 10))
    one_hot_y[np.arange(train_y.size), train_y]
    train_y = one_hot_y
    
    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def deriv_sigmoid(x):
        return sigmoid(x)*(1 - sigmoid(x))
    
    def softmax(x):
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def deriv_softmax(x):
        return softmax(x) * (1 - softmax(x))
    
    #--- Layer
    class Layer:
        #- Constructor
        def __init__(self, in_dim, out_dim, function, deriv_function):
            self.W = np.random.uniform(low=-0.005, high=0.005, size=(in_dim, out_dim)).astype("float32")
            self.b = np.zeros(out_dim).astype("float32")
            self.function = function
            self.deriv_function = deriv_function
            self.u = None
            self.delta = None

        #- Forward Propagation
        def f_prop(self, x):
            self.u = np.dot(x, self.W) + self.b
            self.z = self.function(self.u)
            return self.z

        #- Back Propagation
        def b_prop(self, delta, W):
            self.delta = np.dot(delta, W.T) * self.deriv_function(self.u)
            return self.delta
        
    def f_props(layers, x):
        z = x
        for layer in layers:
            z = layer.f_prop(z)
        return z
    
    def b_props(layers, delta):
        for i, layer in enumerate(layers[::-1]):
            if i == 0:
                layer.delta = delta
            else:
                delta = layer.b_prop(delta, _W)
            _W = layer.W
    
    layers = [Layer(784, 800, sigmoid, deriv_sigmoid),
          Layer(800, 10, softmax, deriv_softmax)]
    
    def train(X, t, eps=0.005):
        #- Forward Propagation
        y = f_props(layers, X)

        #- Cost Function & Delta
        cost = np.sum(-t*np.log(y) - (1 - t)*np.log(1 - y)) # Negative Loglikelihood
        delta = y - t

        #- Back Propagation
        b_props(layers, delta)

        #- Update Parameters
        z = X
        for i, layer in enumerate(layers):
            #print 'z', z
            #print 'layer.delta', layer.delta
            #print 'layer.z', layer.z
            dW = np.dot(z.T, layer.delta) / z.shape[0]
            db = layer.delta.mean(0)
            z = layer.z
            layer.W = layer.W - eps*dW
            layer.b = layer.b - eps*db


        #- Train Cost
        y = f_props(layers, X)
        cost = np.sum(-t*np.log(y) - (1 - t)*np.log(1 - y))
        return cost

    def test(X):
        #- Test Cost
        y = f_props(layers, X)
        #cost = np.sum(-t*np.log(y) - (1 - t)*np.log(1 - y))
        return y
    
    #- Epoch
    train_X = np.array_split(train_X, 56000, axis=0)
    train_y = np.array_split(train_y, 56000, axis=0)
    for epoch in xrange(1):
        #print 'epoch: ', epoch

        for batch_X, batch_y in zip(train_X, train_y):
            cost = train(batch_X, batch_y)
        pred_y = test(test_X)
        pred_y = np.argmax(pred_y, axis=1)
        #print pred_y

    return pred_y

In [35]:
from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

def load_mnist():
    mnist = fetch_mldata('MNIST original')
    mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

    mnist_X = mnist_X / 255.0

    train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

    return (train_X, test_X, train_y, test_y)

def check_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    pred_y = homework(train_X, test_X, train_y)
    return f1_score(test_y, pred_y, average='macro')

if 'homework' in globals():
    result = check_homework()

    print "No Error Occured!"

epoch:  0
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:86: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:65: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:86: RuntimeWarning: invalid value encountered in multiply
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:65: RuntimeWarning: invalid value encountered in multiply
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: invalid value encountered in true_divide


batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
batch x (100, 784)
batch_y (100, 10)
b

KeyboardInterrupt: 

In [46]:
"""from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=0)"""

In [47]:
#homework(train_X, test_X, train_y)

epoch:  0
[4 0 2 ..., 6 3 7]
epoch:  1
[4 0 2 ..., 6 3 7]
epoch:  2
[4 0 2 ..., 6 3 7]
epoch:  3
[4 0 2 ..., 6 3 7]
epoch:  4
[5 0 2 ..., 6 9 8]
epoch:  5
[5 0 2 ..., 6 9 8]
epoch:  6
[5 0 2 ..., 6 9 8]
epoch:  7
[5 0 2 ..., 6 9 8]
epoch:  8
[5 0 2 ..., 6 3 8]
epoch:  9
[5 0 2 ..., 6 3 8]
epoch:  10
[5 0 2 ..., 6 3 8]
epoch:  11
[5 0 2 ..., 6 3 8]
epoch:  12
[5 0 2 ..., 6 3 8]
epoch:  13
[5 0 2 ..., 6 3 8]
epoch:  14
[5 0 2 ..., 6 3 8]
epoch:  15
[4 0 2 ..., 6 3 8]
epoch:  16
[4 0 2 ..., 6 3 8]
epoch:  17
[4 0 2 ..., 6 3 8]
epoch:  18
[4 0 2 ..., 6 3 8]
epoch:  19
[4 0 2 ..., 6 3 8]


array([4, 0, 2, ..., 6, 3, 8])

In [20]:
from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

def load_mnist():
    mnist = fetch_mldata('MNIST original')
    mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

    mnist_X = mnist_X / 255.0
    mnist_X = mnist_X - mnist_X.mean(0, keepdims=True)

    train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

    return (train_X, test_X, train_y, test_y)

def check_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    pred_y = homework(train_X, test_X, train_y)
    return f1_score(test_y, pred_y, average='macro')

if 'homework' in globals():
    result = check_homework()
    print result

    print "No Error Occured!"

0.914
0.919
0.933
0.952
0.969
0.953
0.963
0.964
0.973
0.975
0.979
0.99
0.984
0.99
0.985
0.988
0.985
0.984
0.984
0.987
[7 5 9 ..., 3 1 8]
0.977396599091
No Error Occured!


In [19]:
def homework(train_X, test_X, train_y):
    import numpy as np

    # basic layers, forward and backward pass
    def affine_forward(x, w, b):
        out = np.dot(x, w) + b
        cache = (x, w, b)
        return out, cache
    def affine_backward(dout, cache):
        x, w, b = cache
        dx, dw, db = None, None, None
        dx = np.dot(dout, w.T)
        dw = np.dot(x.T, dout)
        db = dout.sum(0)
        return dx, dw, db
    def relu_forward(x):
        out = None
        out = np.maximum(0, x)
        cache = x
        return out, cache
    def relu_backward(dout, cache):
        dx, x = None, cache
        dx = dout
        dx[x < 0] = 0
        return dx
    def softmax_loss(x, y):
        probs = np.exp(x - np.max(x, axis=1, keepdims=True))
        probs /= np.sum(probs, axis=1, keepdims=True)
        N = x.shape[0]
        loss = -np.sum(np.log(probs[np.arange(N), y])) / N
        dx = probs.copy()
        dx[np.arange(N), y] -= 1
        dx /= N
        return loss, dx


    # combination of affine and relu
    def affine_relu_forward(x, w, b):
        a, fc_cache = affine_forward(x, w, b)
        out, relu_cache = relu_forward(a)
        cache = (fc_cache, relu_cache)
        return out, cache

    def affine_relu_backward(dout, cache):
        fc_cache, relu_cache = cache
        da = relu_backward(dout, relu_cache)
        dx, dw, db = affine_backward(da, fc_cache)
        return dx, dw, db

    class TwoLayerNet(object):
        def __init__(self, input_dim=3*32*32, hidden_dim=100, num_classes=10,
                     weight_scale=1e-3, reg=0.0):
            self.params = {}
            self.reg = reg

            self.params['W1'] = weight_scale * np.random.randn(input_dim, hidden_dim)
            self.params['b1'] = np.zeros(hidden_dim)
            self.params['W2'] = weight_scale * np.random.randn(hidden_dim, num_classes)
            self.params['b2'] = np.zeros(num_classes)
        def loss(self, X, y=None):
            scores = None
            out_1, cache_1 = affine_relu_forward(X, self.params['W1'], self.params['b1'])
            scores, cache_2 = affine_forward(out_1, self.params['W2'], self.params['b2'])
            # If y is None then we are in test mode so just return scores
            if y is None:
                return scores

            loss, grads = 0, {}

            loss, d_scores = softmax_loss(scores, y)
            loss += 1/2.0 * self.reg * (np.sum(self.params['W1'] ** 2) + np.sum(self.params['W2'] ** 2))
            d_out1, grads['W2'], grads['b2'] = affine_backward(d_scores, cache_2)
            dx, grads['W1'], grads['b1'] = affine_relu_backward(d_out1, cache_1)
            grads['W1'] += self.reg * self.params['W1']
            grads['W2'] += self.reg * self.params['W2']

            return loss, grads
        
    class Solver(object):
        def __init__(self, model, train_X, train_y, test_X, batch_size=100, lr=1e-3):
            self.model = model
            self.train_X = train_X
            self.train_y = train_y
            self.test_X = test_X
            self.batch_size = batch_size
            self.loss_history = []
            self.lr = lr
            
        def batch_update(self):
            # Make a minibatch of training data
            num_train = self.train_X.shape[0]
            batch_mask = np.random.choice(num_train, self.batch_size)
            X_batch = self.train_X[batch_mask]
            y_batch = self.train_y[batch_mask]

            # Compute loss and gradient
            loss, grads = self.model.loss(X_batch, y_batch)
            self.loss_history.append(loss)

            # Perform a parameter update
            for p, w in self.model.params.iteritems():
                dw = grads[p]
                self.model.params[p] = w - self.lr * dw
                
        def check_accuracy(self, X, y, num_samples=False, batch_size=100):
            # Maybe subsample the data
            N = X.shape[0]
            if num_samples is not None and N > num_samples:
                mask = np.random.choice(N, num_samples)
                N = num_samples
                X = X[mask]
                y = y[mask]
            # Compute predictions in batches
            num_batches = N // batch_size
            if N % batch_size != 0:
                num_batches += 1
            y_pred = []
            for i in xrange(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                scores = self.model.loss(X[start:end])
                y_pred.append(np.argmax(scores, axis=1))
            y_pred = np.hstack(y_pred)
            acc = np.mean(y_pred == y)

            return acc
        
        def train(self):
            for epoch in xrange(20):
                for i in xrange(560):
                    self.batch_update()
                train_acc = self.check_accuracy(self.train_X, self.train_y, num_samples=1000)
                #print train_acc
            
        def predict(self):
            scores = self.model.loss(self.test_X)
            pred_y = np.argmax(scores, axis=1)
            return pred_y
        
    model = TwoLayerNet(input_dim=784, hidden_dim=800, num_classes=10)
    solver = Solver(model, train_X, train_y, test_X, batch_size=100, lr=1e-1)
    solver.train()
    pred_y = solver.predict()
    #print pred_y
    
    return pred_y